# 第 1 步: 获取过去指定数据（使用 https://cds.climate.copernicus.eu/api 数据源）

In [1]:
import cdsapi
import logging
import xarray as xr
from pathlib import Path
from datetime import datetime, timezone, date
from zoneinfo import ZoneInfo
from typing import Dict, Set, List
import zipfile

# --- 1. 从我们的配置文件中导入所有常量 ---
# 因为项目是以可编辑模式安装的 (-e .), 我们可以直接从包中导入
from chromasky_toolkit import config



# --- 2. 定义本次运行的特定参数 ---
# 注意：像目标日期这样的变量不适合放在 config.py 中，因为它每次运行都可能不同。
# 把它留在 Notebook 中是正确的做法。
TARGET_DATE_STR = "2025-07-18" 

# --- 3. 设置日志 (这部分也可以保留在 Notebook 中) ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger("ERA5_Downloader")

# --- 4. 验证导入的配置 (可选) ---
print("\n--- 使用的配置 ---")
print(f"CDS API Key Loaded: {'Yes' if config.CDS_API_KEY else 'No'}")
print(f"Extraction Area (North): {config.AREA_EXTRACTION['north']}")
print(f"Data Directory: {config.DATA_DIR}")
print(f"Target Date for this run: {TARGET_DATE_STR}")

⚠️ Config: 未找到 .env 文件于 /Users/zhangchao/Documents/Code/github/chromasky-toolkit/src/.env

--- 使用的配置 ---
CDS API Key Loaded: Yes
Extraction Area (North): 54.0
Data Directory: /Users/zhangchao/Documents/Code/github/chromasky-toolkit/src/data
Target Date for this run: 2025-07-18


In [2]:
def get_required_utc_dates_and_hours(target_local_date: date) -> Dict[str, Set[int]]:
    """
    根据目标本地日期和预设的日出/日落时间，计算出需要下载的UTC日期和小时。
    这是处理跨时区、跨零点问题的关键。
    """
    local_tz = ZoneInfo(config.LOCAL_TZ)
    all_event_times = config.SUNRISE_EVENT_TIMES + config.SUNSET_EVENT_TIMES
    utc_date_hours: Dict[str, Set[int]] = {}

    logger.info(f"为本地日期 {target_local_date} 计算所需的 UTC 时间...")
    for time_str in all_event_times:
        try:
            local_dt = datetime.combine(target_local_date, datetime.strptime(time_str, '%H:%M').time(), tzinfo=local_tz)
            utc_dt = local_dt.astimezone(timezone.utc)
            
            utc_date_str = utc_dt.strftime('%Y-%m-%d')
            if utc_date_str not in utc_date_hours:
                utc_date_hours[utc_date_str] = set()
            utc_date_hours[utc_date_str].add(utc_dt.hour)
            # logger.debug(f"本地时间 {local_dt} -> UTC 时间 {utc_dt}")
        except Exception as e:
            logger.error(f"处理时间 '{time_str}' 时出错: {e}")

    logger.info(f"计算完成的 UTC 请求信息: {utc_date_hours}")
    return utc_date_hours


def download_era5_data(target_local_date: date):
    """
    为指定的本地日期下载 ERA5 再分析数据。
    *** 新版本：增加了自动解压 ZIP 文件的功能 ***
    """
    if not (config.CDS_API_KEY):
        logger.error("❌ CDS API 配置未找到，无法继续下载。")
        return None

    output_dir = config.ERA5_DATA_DIR / target_local_date.strftime('%Y-%m-%d')
    output_dir.mkdir(parents=True, exist_ok=True)
    final_output_file = output_dir / "era5_data.nc"
    
    # 定义一个临时的下载文件名
    temp_download_path = output_dir / "temp_download"

    logger.info(f"--- [ERA5] 函数内部检查路径: {final_output_file} ---")
    
    if final_output_file.exists():
        logger.info(f"✅ (函数内部) 最终文件 '{final_output_file.name}' 已存在，跳过。")
        return final_output_file

    required_utc_info = get_required_utc_dates_and_hours(target_local_date)
    if not required_utc_info:
        logger.warning("未能计算出任何需要下载的UTC日期和小时。")
        return None

    years, months, days, hours = set(), set(), set(), set()
    for utc_date_str, hours_set in required_utc_info.items():
        dt_obj = datetime.strptime(utc_date_str, '%Y-%m-%d')
        years.add(f"{dt_obj.year}")
        months.add(f"{dt_obj.month:02d}")
        days.add(f"{dt_obj.day:02d}")
        hours.update([f"{h:02d}:00" for h in hours_set])
    
    request_params = {
        'year': sorted(list(years)),
        'month': sorted(list(months)),
        'day': sorted(list(days)),
        'time': sorted(list(hours)),
    }
    
    logger.info("将为以下参数发起下载请求:")
    for key, value in request_params.items():
        logger.info(f"  > {key.capitalize()}: {value}")

    c = cdsapi.Client(timeout=600, quiet=False, url="https://cds.climate.copernicus.eu/api", key=config.CDS_API_KEY)
    area_bounds = [config.AREA_EXTRACTION[k] for k in ["north", "west", "south", "east"]]
    logger.info(f"请求区域边界: {area_bounds}")
    try:
        # 1. 下载到临时的文件，而不是直接命名为 .nc
        logger.info("正在向 CDS 服务器发送请求...")
        c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf', # 即使请求 netcdf, 服务器也可能返回 zip
                'variable': [
                    "high_cloud_cover", "medium_cloud_cover", "low_cloud_cover", 
                    "total_cloud_cover", "total_precipitation", "surface_pressure",
                    "2m_temperature", "2m_dewpoint_temperature"
                ],
                'area': area_bounds,
                **request_params
            },
            str(temp_download_path)
        )
        logger.info(f"✅ 临时文件已成功下载到: {temp_download_path}")

        # 2. 检查下载的是 ZIP 还是直接就是 NC
        if zipfile.is_zipfile(temp_download_path):
            logger.info("检测到下载文件为 ZIP 压缩包，开始解压...")
            with zipfile.ZipFile(temp_download_path, 'r') as zip_ref:
                # 寻找解压出来的 .nc 文件
                nc_files_in_zip = [f for f in zip_ref.namelist() if f.endswith('.nc')]
                if not nc_files_in_zip:
                    raise FileNotFoundError("ZIP 包中未找到任何 .nc 文件！")
                
                # 解压第一个找到的 .nc 文件
                source_nc_path = zip_ref.extract(nc_files_in_zip[0], path=output_dir)
                logger.info(f"已解压出 NetCDF 文件: {source_nc_path}")
                
                # 将解压出的文件重命名为我们最终想要的名字
                Path(source_nc_path).rename(final_output_file)
                logger.info(f"已将文件重命名为: {final_output_file}")
        else:
            # 如果不是 ZIP，说明直接下载的就是 NetCDF 文件
            logger.info("检测到下载文件为 NetCDF，直接重命名。")
            temp_download_path.rename(final_output_file)

        return final_output_file

    except Exception as e:
        logger.error(f"❌ 下载或解压过程中发生严重错误: {e}", exc_info=True)
        return None
    finally:
        # 4. 清理临时文件
        if temp_download_path.exists():
            temp_download_path.unlink()

In [3]:
import xarray as xr
from pathlib import Path

def generate_analysis_report(ds: xr.Dataset) -> str:
    """
    根据 xarray.Dataset 对象，生成一份详细的 Markdown 格式的分析报告。

    Args:
        ds: 已打开的 xarray Dataset 对象。

    Returns:
        一个包含完整报告的 Markdown 格式字符串。
    """
    report_lines = []
    source_file = Path(ds.encoding.get("source", "N/A")).name
    
    report_lines.append(f"# NetCDF 文件分析报告: `{source_file}`")
    report_lines.append(f"报告生成时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # 1. 全局属性
    report_lines.append("\n## 1. 全局属性 (Global Attributes)")
    if ds.attrs:
        for key, value in ds.attrs.items():
            report_lines.append(f"- **{key}:** `{value}`")
    else:
        report_lines.append("- (无全局属性)")

    # 2. 维度信息
    report_lines.append("\n## 2. 维度信息 (Dimensions)")
    for dim_name, dim_size in ds.sizes.items():
        report_lines.append(f"- `{dim_name}`: **{dim_size}**")

    # 3. 坐标变量
    report_lines.append("\n## 3. 坐标变量 (Coordinates)")
    for coord_name in ds.coords:
        coord_var = ds[coord_name]
        report_lines.append(f"\n### 坐标: `{coord_name}`")
        report_lines.append(f"- **维度:** `{coord_var.dims}`")
        report_lines.append(f"- **类型:** `{coord_var.dtype}`")
        if 'units' in coord_var.attrs:
            report_lines.append(f"- **单位:** {coord_var.attrs.get('long_name', '')} (`{coord_var.attrs.get('units', '')}`)")
        if 'time' in coord_name:
            report_lines.append(f"- **时间范围 (UTC):** `{coord_var.min().values}` 到 `{coord_var.max().values}`")
        elif 'lat' in coord_name or 'lon' in coord_name:
            report_lines.append(f"- **范围:** `{coord_var.min().values:.2f}` 到 `{coord_var.max().values:.2f}`")

    # 4. 数据变量
    report_lines.append("\n## 4. 数据变量 (Data Variables)")
    for var_name in ds.data_vars:
        var = ds[var_name]
        report_lines.append(f"\n### 变量: `{var_name}`")
        report_lines.append(f"- **维度 (shape):** `{var.dims}` -> `{var.shape}`")
        report_lines.append(f"- **数据类型:** `{var.dtype}`")
        if var.attrs:
            report_lines.append("- **属性:**")
            for attr_key, attr_value in var.attrs.items():
                report_lines.append(f"  - **{attr_key}:** `{attr_value}`")
        if var.ndim > 0:
            indexers = {dim: 0 for dim in var.dims}
            sample_value = var.isel(**indexers).values
            report_lines.append(f"- **抽样值 (在索引 {indexers}):** `{sample_value:.4f}`")
            
    return "\n".join(report_lines)

In [4]:
# --- 1. 定义本次运行的目标日期 ---
# (这个变量定义可以保留在之前的单元格，这里重申以保证逻辑完整)
target_date_obj = datetime.strptime(TARGET_DATE_STR, "%Y-%m-%d").date()

# --- 2. 检查文件是否存在 & 执行下载 ---
logger.info(f"===== 开始处理本地日期: {target_date_obj} =====")
expected_file_path = config.ERA5_DATA_DIR / target_date_obj.strftime('%Y-%m-%d') / "era5_data.nc"

if expected_file_path.exists() and expected_file_path.stat().st_size < 1024:
    logger.warning(f"发现一个之前下载失败的小文件，正在删除: {expected_file_path}")
    expected_file_path.unlink()

if expected_file_path.exists():
    logger.info(f"✅ NC 文件已存在，跳过下载: {expected_file_path}")
    downloaded_file_path = expected_file_path
else:
    logger.info(f"NC 文件不存在，开始执行下载流程...")
    downloaded_file_path = download_era5_data(target_date_obj)

# --- 3. 生成并保存分析报告 ---
if downloaded_file_path and downloaded_file_path.exists():
    
    report_file_path = downloaded_file_path.with_suffix('.md')
    
    # 检查报告是否已存在，如果已存在则跳过分析
    if report_file_path.exists():
        logger.info(f"✅ 分析报告已存在，跳过分析: {report_file_path}")
    else:
        logger.info(f"正在为 {downloaded_file_path.name} 生成分析报告...")
        try:
            with xr.open_dataset(downloaded_file_path, engine="netcdf4") as ds:
                # 调用新函数生成报告内容
                report_content = generate_analysis_report(ds)
                
                # 将报告内容写入文件
                report_file_path.write_text(report_content, encoding='utf-8')
                
                logger.info(f"✅ 分析报告已成功保存到: {report_file_path}")

        except Exception as e:
            logger.error(f"❌ 分析或保存报告时发生错误: {e}", exc_info=True)

elif not downloaded_file_path:
    logger.error("\n😢 下载流程失败，无法生成分析报告。")

2025-08-07 14:25:37,735 - INFO - ===== 开始处理本地日期: 2025-07-18 =====
2025-08-07 14:25:37,736 - INFO - NC 文件不存在，开始执行下载流程...
2025-08-07 14:25:37,736 - INFO - --- [ERA5] 函数内部检查路径: /Users/zhangchao/Documents/Code/github/chromasky-toolkit/src/data/raw/era5/2025-07-18/era5_data.nc ---
2025-08-07 14:25:37,737 - INFO - 为本地日期 2025-07-18 计算所需的 UTC 时间...
2025-08-07 14:25:37,737 - INFO - 计算完成的 UTC 请求信息: {'2025-07-17': {20, 21, 22, 23}, '2025-07-18': {0, 10, 11, 12, 13}}
2025-08-07 14:25:37,738 - INFO - 将为以下参数发起下载请求:
2025-08-07 14:25:37,738 - INFO -   > Year: ['2025']
2025-08-07 14:25:37,738 - INFO -   > Month: ['07']
2025-08-07 14:25:37,738 - INFO -   > Day: ['17', '18']
2025-08-07 14:25:37,739 - INFO -   > Time: ['00:00', '10:00', '11:00', '12:00', '13:00', '20:00', '21:00', '22:00', '23:00']
2025-08-07 14:25:38,669 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-08-07 14:25:38,669 - INFO - [2024-09-26T00:00:00] Watch ou

c6e34e4ad10036b0dce8f7d070cb307c.zip:   0%|          | 0.00/12.1M [00:00<?, ?B/s]

2025-08-07 14:26:03,232 - INFO - ✅ 临时文件已成功下载到: /Users/zhangchao/Documents/Code/github/chromasky-toolkit/src/data/raw/era5/2025-07-18/temp_download
2025-08-07 14:26:03,234 - INFO - 检测到下载文件为 ZIP 压缩包，开始解压...
2025-08-07 14:26:03,260 - INFO - 已解压出 NetCDF 文件: /Users/zhangchao/Documents/Code/github/chromasky-toolkit/src/data/raw/era5/2025-07-18/data_stream-oper_stepType-instant.nc
2025-08-07 14:26:03,261 - INFO - 已将文件重命名为: /Users/zhangchao/Documents/Code/github/chromasky-toolkit/src/data/raw/era5/2025-07-18/era5_data.nc
2025-08-07 14:26:03,262 - INFO - 正在为 era5_data.nc 生成分析报告...
2025-08-07 14:26:03,543 - INFO - ✅ 分析报告已成功保存到: /Users/zhangchao/Documents/Code/github/chromasky-toolkit/src/data/raw/era5/2025-07-18/era5_data.md
